In [1]:
# data manipulation
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# plotly 
import plotly.express as px
import plotly.graph_objects as go

# dashboards
import dash
from jupyter_dash import JupyterDash #for running dash in jupyer notebook
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc #for dashboard theme
from datetime import date

#optimization
from gurobipy import GRB,Model

<ipython-input-1-f54c80547a8b>:13: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-f54c80547a8b>:14: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-f54c80547a8b>:15: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [2]:
# read restaurant menu information
menu_raw = pd.read_csv('CleanedRestaurant.csv')
menu = menu_raw.rename(columns={'menu':'food','calories':'cal'})
menu.head()


,brand,food,cal,fat,cho,sodium,charbo,sugar,protein
0,7-Eleven,5 React Fruit Slurpee,66.0,NaN,0.0,6.0,18.0,18.0,0.0
1,7-Eleven,Black Iced Coffee,5.0,NaN,0.0,25.0,1.0,NaN,0.0
2,7-Eleven,Blue Lightning Blast Slurpee,70.0,NaN,0.0,20.0,18.0,18.0,0.0
3,7-Eleven,Cherry Orange Blitz Slurpee,110.0,NaN,0.0,25.0,28.0,27.0,0.0
4,7-Eleven,Coca Cola Classic Slurpee,179.0,NaN,0.0,16.0,49.0,49.0,0.0


In [3]:
# read exercise type information
df = pd.read_csv('exercise_data.csv')
exercise_list=[{'label': i, 'value': i} for i in list(df['Category'].unique())]

In [4]:
# parsing image
import base64

In [5]:
# import and encode images
image_filename1 = 'optimal_ex.jpg' 
image_filename2 = 'single_ex.png' 
image_filename3 = 'junk.png'

encoded_image1 = base64.b64encode(open(image_filename1, 'rb').read())
encoded_image2 = base64.b64encode(open(image_filename2, 'rb').read())
encoded_image3 = base64.b64encode(open(image_filename3, 'rb').read())


In [6]:
# using Jupyter Notebook
app = JupyterDash(__name__)

# list brand1 and brand2
brand1=[{'label': i, 'value': i} for i in list(menu['brand'].unique())]
brand2=[{'label': i, 'value': i} for i in list(menu['brand'].unique())]

# set the tab style
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold',
    'font-family': '"Open Sans", verdana, arial, sans-serif'
}

# set the clicked tab style
tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#478af5',
    'color': 'white',
    'padding': '6px',
    'font-family': '"Open Sans", verdana, arial, sans-serif'
}

# create the basic layout
app.layout = html.Div([
    # head of three different tabs
    html.Div([
        html.Div([
            html.H1('Fast Food Nutrition Calculator',
                    style={'color': '#f04a30',
                        'fontSize': '45px',
                        'font-family': '"Open Sans", verdana, arial, sans-serif'}),
            
            html.H4('Helps you figure out your fact for your meals',
                    style={'color': '#fc9c26',
                        'fontSize': '25px',
                        'font-family': '"Open Sans", verdana, arial, sans-serif'})],
                        style = {'width':'50%','display':'inline-block'}),
        # add a image to the head
        html.Div([
                    html.Br()
                    ],style={'width':'20%','display':'inline-block'}),
                    html.Div([html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image3.decode()),height=150,width=360
                    )],style={'width':'28%','display':'inline-block'})
    ],style={'width':'100%'}),


    # Main body of the dash: 3 tabs
    dcc.Tabs([
        # content of first tab
       dcc.Tab(children=[
            # layer 1: whole layer
            html.Div([
                html.Br(),
                html.Br(),
                # left part of the dash: select brands and menus
                html.Div([
                        html.Div([
                        html.H3("Select Restaurants and meal items!")],style={"margin":"10px",'font-family': '"Gill Sans", sans-serif'}),
                        html.Br(),
                        html.Br(),
                        html.Br(),
                        html.Div([
                                html.Label("Choose a Restaurants and meal items:"),
                                html.Br(),
                                # callback 1, brand 1
                                dcc.Dropdown(
                                        id='brand1',
                                        value='No choice',
                                        placeholder='Please type name of restaurant1'),
                                html.Br(),
                                html.Div([
                                    html.Br()
                                ],style={"width": "10%","display":"inline-block"}),
                                html.Div([
                                    #callback 2, brand 1 food 1
                                    dcc.Dropdown(
                                        id='brand1food1',
                                        placeholder='Please type a food name in restaurant1',
                                        value = 'No choice'),
                                
                                    #callback 3, brand 1 food 2
                                    dcc.Dropdown(
                                        id='brand1food2',
                                        placeholder='Please type a food name in restaurant1',
                                        value = 'No choice'),
                                
                                    #callback 4, brand 1 food 3
                                    dcc.Dropdown(
                                        id='brand1food3',
                                        placeholder='Please type a food name in restaurant1',
                                        value = 'No choice'),
                                
                                    #callback 5, brand 1 food 4
                                    dcc.Dropdown(
                                        id='brand1food4',
                                        placeholder='Please type a food name in restaurant1',
                                        value = 'No choice')],style={"width": "85%","display":"inline-block"}),

                                html.Br(),
                                html.Br(),


                                # callback 6, brand 2
                                dcc.Dropdown(
                                        id='brand2',
                                        placeholder='Please type name of restaurant2',
                                        value = 'No choice'),
                                html.Br(),
                                html.Div([
                                    html.Br()
                                ],style={"width": "10%","display":"inline-block"}),
                                html.Div([
                                    #callback 7, brand 2 food 1
                                    dcc.Dropdown(
                                        id='brand2food1',
                                        placeholder='Please type a food name in restaurant2',
                                        value = 'No choice'),
                                
                                    #callback 8, brand 2 food 2
                                    dcc.Dropdown(
                                        id='brand2food2',
                                        placeholder='Please type a food name in restaurant2',
                                        value = 'No choice'),
                                
                                    #callback 9, brand 2 food 3
                                    dcc.Dropdown(
                                        id='brand2food3',
                                        placeholder='Please type a food name in restaurant2',
                                        value = 'No choice'),
                                
                                    #callback 10, brand 2 food 4
                                    dcc.Dropdown(
                                        id='brand2food4',
                                        placeholder='Please type a food name in restaurant2',
                                        value = 'No choice')],style={"width": "85%","display":"inline-block"}),
                                html.Br(),
                                html.Br(),
                                html.P("**Same restaurant can be selected"),
                                html.P("**You can select same meal item in a restaurant"),
                                html.P("**Meal item from different restaurant cannot be selected in other restaurant"),
                                html.Br()
                        ],style={'margin':"10px",'font-family': '"Gill Sans", sans-serif'}),
                    
            ],
            style={'border-radius': '10px',"width": "20%","display":"inline-block","border":"2px lightgrey solid",'box-shadow': '2px 5px 5px 1px grey','background-color': 'white'}),
            
                # for beauty, insert a 4% block
                html.Div([
                                    html.Br(),
                                ],
                            style={"width": "4%","display":"inline-block"}),

                # for the right part (including two main parts, four subparts)
                html.Div([
                    # the following code represents the top main part of this subparts
                    html.Div([

                        # the total calories number, callback 11
                        html.Div([
                            html.Div([html.H3("Total Calories")],style={"margin":"10px",'font-family': '"Gill Sans", sans-serif'}),
                            html.Br(),
                            html.Br(),
                            html.H1(id = "total-calories",
                                    children = "0",
                                    style={'color': '#f04a30', 'fontSize': '130px', "margin":"20px"}),
                            html.Br(),
                            html.Div([
                            html.P("**Sum of calories of all chosen food items from selected restaurants ")],style = {'font-family': '"Gill Sans", sans-serif'}),
                        ],style={'border-radius': '10px',"width": "28%","display":"inline-block","border":"2px lightgrey solid",'box-shadow': '2px 5px 5px 1px grey','background-color': 'white'}),

                        # for beauty, add a block
                        html.Div([
                            html.Br(),
                        ],style={"width": "3%","display":"inline-block"}),

                        # table and pie chart for total calories breakdown
                        html.Div([
                            html.Div([html.H3("Total Calories Breakdown")],style={"margin":"10px",'font-family': '"Gill Sans", sans-serif'}),
                            # the table for each kind of food, callback 12
                            html.Div([
                                dcc.Graph(style={'height':280,'width':450},
                                id= "table_for_each"),
                            ],style={"width": "55%","display":"inline-block","margin":"10px",'font-family': '"Gill Sans", sans-serif'}),
                            
                            # for beauty, add a block
                            html.Div([
                                html.Br(),
                                ],style={"width": "5%","display":"inline-block"}),

                            # the pie chart for each kind of food, callback 13
                            html.Div([
                                dcc.Graph(style={'height':280,'width':280},
                                id= "pie_chart_for_each_choice")
                            ],style={"width": "30%","display":"inline-block"})
                        ],style={'border-radius': '10px',"width": "68%","display":"inline-block","border":"2px lightgrey solid",'box-shadow': '2px 5px 5px 1px grey','background-color': 'white','font-family': '"Gill Sans", sans-serif'})


                    ],style={"width": "100%","display":"inline-block"}),

                    # add some block lines, for beauty
                    html.Br(),
                    html.Br(),


                    # the following code represents the buttom main part of this subparts
                    html.Div([

                        # stack bar chart of the nutritions , callback 14
                        html.Div([
                            dcc.Graph(style={'height':290,'width':500},
                            id= "g",hoverData={'data':[{'nutritiondfnew':'name'}]}),
                            html.Div([html.H3("Other Nutritions Breakdown")],style={"margin":"10px",'font-family': '"Gill Sans", sans-serif'})
                        ],style={'border-radius': '10px',"width": "45%","display":"inline-block","border":"2px lightgrey solid",'box-shadow': '2px 5px 5px 1px grey','background-color': 'white','font-family': '"Gill Sans", sans-serif'}),

                        html.Div([
                            html.Br()
                        ],style={"width": "5%","display":"inline-block"}),


                        # the table of recommended amount of nutritions, callback 15
                        html.Div(
                            [html.Div([html.H3("Actual Vs Recommended Nutritions")],style={"margin":"10px",'font-family': '"Gill Sans", sans-serif'}),
                            dcc.Graph(style={'height':240,'width':550},
                            id = "recommendation_amount"),
                            html.P("**Difference tells you about how far you are from the daily recommended amount!")]
                        ,style={'border-radius': '10px',"width": "45%","display":"inline-block","border":"2px lightgrey solid",'box-shadow': '2px 5px 5px 1px grey','background-color': 'white','font-family': '"Gill Sans", sans-serif'})

                    ],style={"width": "100%","height": "120px","display":"inline-block",'font-family': '"Gill Sans", sans-serif'})

                        
                                ],
                            style={"width": "75%","display":"inline-block"})
        ],style={'background-color':'#ffcccb'}),
            html.Div([
                html.Br()
            ],style={'background-color':'#ffcccb'})
            
            

               ],label='Nutrition Calculator',style=tab_style, selected_style=tab_selected_style),
           
           
           
        #the content of second tab, mainly talk about the optimization of the exercises
        dcc.Tab(children=[
            # the top box of the second tab, choose the exercise and weight
            html.Div([
                html.Br(),
                html.Div([
                    html.Div([html.H2("Help us know your favorite workouts!")],style={"margin":"20px"}),
                    html.Br(),
                    html.Div([
                        html.Br()
                        ], style={'width': '2%','display':'inline-block'}),
                    html.Div([
                        "Exercise #1",
                        dcc.Dropdown(
                            id='exercise_1',
                            placeholder='Please choose your first exercise'
                        )], style={'width': '14%','display':'inline-block'}),
                    html.Div([
                        html.Br()
                        ], style={'width': '2%','display':'inline-block'}),
                    html.Div([
                        "Exercise #2",
                        dcc.Dropdown(
                            id='exercise_2',
                            placeholder='Please choose your second exercise',
                        )], style={'width': '14%','display':'inline-block'}),
                    html.Div([
                        html.Br()
                        ], style={'width': '2%','display':'inline-block'}),
                    html.Div([
                        "Exercise #3",
                        dcc.Dropdown(
                            id='exercise_3',
                            placeholder='Please choose your third exercise'
                        )], style={'width': '14%','display':'inline-block'}),
                    html.Div([
                        html.Br()
                        ], style={'width': '2%','display':'inline-block'}),
                    html.Div([
                        "Exercise #4",
                        dcc.Dropdown(
                            id='exercise_4',
                            placeholder='Please choose your fourth exercise',
                        )], style={'width': '14%','display':'inline-block'}),
                    html.Div([
                        html.Br()
                        ], style={'width': '2%','display':'inline-block'}),
                    html.Div([
                        "Exercise #5",
                        dcc.Dropdown(
                            id='exercise_5',
                            placeholder='Please choose your fifth exercise'
                        )], style={'width': '14%','display':'inline-block'}),
                    html.Div([
                        html.Br()
                        ], style={'width': '2%','display':'inline-block'}),
                    html.Div([
                            "Weight (in kg)",
                            dcc.Dropdown(
                                id='weight',
                                value = 75,
                                options=[{'label': i, 'value': i} for i in range(40,101)],
                                placeholder='Please choose your weight in kg',
                            )], style={'width': '14%','display':'inline-block'}),
                    html.Div([
                        html.Br()
                    ])
                ],style={'font-family': '"Gill Sans", sans-serif','border-radius': '10px',"border":"2px lightgrey solid",'box-shadow': '2px 5px 5px 1px grey',"background-color":"white"}),
                
                html.Br(),

                # the second part of tab 2: use gurobi optimization to choose the optimal exercise and deliver information with table and pie chart
                html.Div([
                    html.Div([
                    html.Div([
                    html.H2("Opitmal Workout")],style={'margin':"20px",'font-family': '"Gill Sans", sans-serif'}),  ## change to H2
                    html.Br(),
                    html.Div([
                            dcc.Graph(style={'height':260,'width':400},
                            id= "table_for_each2"),
                            ],style={"width": "44%","display":"inline-block","margin":"20px"}),
                    html.Div([
                        html.Br()
                        ], style={'width': '5%','display':'inline-block'}),
                    html.Div([
                            dcc.Graph(style={'height':260,'width':280},
                            id= "pie_chart_for_each_choice2")
                            ],style={"width": "35%","display":"inline-block","margin":"20px"})
                ],style={"width": "57%","display":"inline-block"}),
                    html.Div([
                        html.Br()
                    ],style={'width':'3%','display':'inline-block'}),
                    html.Div([html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image1.decode()),height=380,width=420
                    )],style={'width':'38%','display':'inline-block'})
                ],style={"width":"80%",'border-radius': '10px',"border":"2px lightgrey solid",'box-shadow': '2px 5px 5px 1px grey',"background-color":"white"}),
                
                html.Br(),

                # the third part of tab 2: calculate the burning of single exercise and deliver information with table and bar chart
                html.Div([
                html.Div([
                    html.Br()
                ],style={'width':"19%","display":'inline-block'}),
                html.Div([
                    html.Div([
                        html.Br()
                    ],style={"width":"1%",'display':"inline-block"}),
                    html.Div([html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image2.decode()),height=380,width=440
                    )],style={'width':'38%','display':'inline-block'}),
                    html.Div([
                        html.Div([html.H2("Number of Hours to burn All Calories from only One Workout")],style={"margin":"10px",'font-family': '"Gill Sans", sans-serif'}),
                        html.Br(),
                        html.Div([
                                dcc.Graph(style={'height':260,'width':400},
                                id= "table_for_single2"),
                                ],style={"width": "44%","display":"inline-block","margin":"10px"}),
                        html.Div([
                            html.Br()
                            ], style={'width': '5%','display':'inline-block'}),
                        html.Div([
                                dcc.Graph(style={'height':260,'width':350},
                                id= "bar_chart_for_each2")
                                ],style={"width": "35%","display":"inline-block","margin":"10px"})
                    ],style={"width": "60%","display":"inline-block"})
],style={"width":"80%",'border-radius': '10px',"border":"2px lightgrey solid",'box-shadow': '2px 5px 5px 1px grey',"display":'inline-block',"background-color":"white"})],style={'width':"100%"})
            ],style={'background-color':'#ffcccb'}),
            html.Div([
                html.Br()
            ],style={'background-color':'#ffcccb'})
        ], label='Workout Plan',style=tab_style, selected_style=tab_selected_style),


        # the following content is for the tab3
        dcc.Tab(children=[
                # first part: choose the hours can exercise
                html.Div([
                    html.Div([
                    html.H2("Select the number of hours you can workout"),
                    dcc.Dropdown(
                                value = 3,
                                id='exercise_hour',
                                options=[{'label': i, 'value': i} for i in [m for m in np.arange(0,18,0.5)]],
                                placeholder='Please choose hours to exercise'
                            )], style={'width': '40%','display':'inline-block','font-family': '"Gill Sans", sans-serif'}),
                    html.Div([
                        html.Br()
                    ],style={"width":"20%",'display':"inline-block"}),
                    html.Div([
                        html.H1("Just Do It NOW!")
                    ],style={'width':'40%','display':'inline-block',
                        'fontSize': '30px',
                        'font-family': '"Open Sans", verdana, arial, sans-serif','color':'red'})
                            ]),
                html.Br(),

                # second part, give additional info by table
                html.Div([
                    html.H3("Additional Information"),
                    html.Br(),
                    html.Div([
                            dcc.Graph(
                            id= "food_exercise"),
                            ])
                ],style = {'font-family': '"Gill Sans", sans-serif'})]
        , label='Additional Information',style=tab_style, selected_style=tab_selected_style)
    ]),
    html.Div([
        html.H6('THIS DASHBOARD IS CREATED BY ZHIYU ZHANG. SOURCE CODE CAN BE FOUND ON GITHUB: ZhiyuZhang803')
    ],style={'background-color':'white'})
])


# brand 1 callback
@app.callback(
    Output("brand1", "options"),
    Input("brand1", "search_value")
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate
    return [br1 for br1 in brand1 if search_value in br1["label"]]

#brand 1 food 1 call back
@app.callback(
    Output("brand1food1", "options"),
    Input("brand1food1", "search_value"),
    Input("brand1","value")
)
def update_options(search_value,brand1):
    brand1menu = menu[menu['brand']==brand1]['food']
    brand1food1=[{'label': i, 'value': i} for i in list(brand1menu)]
    if not search_value:
        raise PreventUpdate
    return [br1f1 for br1f1 in brand1food1 if search_value in br1f1["label"]]

#brand 1 food 2 call back
@app.callback(
    Output("brand1food2", "options"),
    Input("brand1food2", "search_value"),
    Input("brand1","value")
)
def update_options(search_value,brand1):
    brand1menu = menu[menu['brand']==brand1]['food']
    brand1food2=[{'label': i, 'value': i} for i in list(brand1menu)]
    if not search_value:
        raise PreventUpdate
    return [br1f2 for br1f2 in brand1food2 if search_value in br1f2["label"]]

#brand 1 food 3 call back
@app.callback(
    Output("brand1food3", "options"),
    Input("brand1food3", "search_value"),
    Input("brand1","value")
)
def update_options(search_value,brand1):
    brand1menu = menu[menu['brand']==brand1]['food']
    brand1food3=[{'label': i, 'value': i} for i in list(brand1menu)]
    if not search_value:
        raise PreventUpdate
    return [br1f3 for br1f3 in brand1food3 if search_value in br1f3["label"]]

#brand 1 food 4 call back
@app.callback(
    Output("brand1food4", "options"),
    Input("brand1food4", "search_value"),
    Input("brand1","value")
)
def update_options(search_value,brand1):
    brand1menu = menu[menu['brand']==brand1]['food']
    brand1food4=[{'label': i, 'value': i} for i in list(brand1menu)]
    if not search_value:
        raise PreventUpdate
    return [br1f4 for br1f4 in brand1food4 if search_value in br1f4["label"]]


# brand 2 callback
@app.callback(
    Output("brand2", "options"),
    Input("brand2", "search_value")
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate
    return [br2 for br2 in brand2 if search_value in br2["label"]]

#brand 2 food 1 call back
@app.callback(
    Output("brand2food1", "options"),
    Input("brand2food1", "search_value"),
    Input("brand2","value")
)
def update_options(search_value,brand2):
    brand2menu = menu[menu['brand']==brand2]['food']
    brand2food1=[{'label': i, 'value': i} for i in list(brand2menu)]
    if not search_value:
        raise PreventUpdate
    return [br2f1 for br2f1 in brand2food1 if search_value in br2f1["label"]]

#brand 2 food 2 call back
@app.callback(
    Output("brand2food2", "options"),
    Input("brand2food2", "search_value"),
    Input("brand2","value")
)
def update_options(search_value,brand2):
    brand2menu = menu[menu['brand']==brand2]['food']
    brand2food2=[{'label': i, 'value': i} for i in list(brand2menu)]
    if not search_value:
        raise PreventUpdate
    return [br2f2 for br2f2 in brand2food2 if search_value in br2f2["label"]]

#brand 2 food 3 call back
@app.callback(
    Output("brand2food3", "options"),
    Input("brand2food3", "search_value"),
    Input("brand2","value")
)
def update_options(search_value,brand2):
    brand2menu = menu[menu['brand']==brand2]['food']
    brand2food3=[{'label': i, 'value': i} for i in list(brand2menu)]
    if not search_value:
        raise PreventUpdate
    return [br2f3 for br2f3 in brand2food3 if search_value in br2f3["label"]]

#brand 2 food 4 call back
@app.callback(
    Output("brand2food4", "options"),
    Input("brand2food4", "search_value"),
    Input("brand2","value")
)
def update_options(search_value,brand2):
    brand2menu = menu[menu['brand']==brand2]['food']
    brand2food4=[{'label': i, 'value': i} for i in list(brand2menu)]
    if not search_value:
        raise PreventUpdate
    return [br2f4 for br2f4 in brand2food4 if search_value in br2f4["label"]]

# callback11: calculate the total calories
@app.callback(
    Output("total-calories", "children"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def calculate_cal(b1,b2,b3,b4,b5,b6,b7,b8):
    total_cal = menu[menu['food']==b1].iloc[0,2]+ menu[menu['food']==b2].iloc[0,2]+ \
                menu[menu['food']==b3].iloc[0,2]+ menu[menu['food']==b4].iloc[0,2]+ \
                menu[menu['food']==b5].iloc[0,2]+ menu[menu['food']==b6].iloc[0,2]+ \
                menu[menu['food']==b7].iloc[0,2]+ menu[menu['food']==b8].iloc[0,2]
    return total_cal

# callback13: pie chart to break down the total calories
@app.callback(
    Output("pie_chart_for_each_choice", "figure"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def draw_pie_graph(b1,b2,b3,b4,b5,b6,b7,b8):
    cal1 = menu[menu['food']==b1].iloc[0,2]
    cal2 = menu[menu['food']==b2].iloc[0,2]
    cal3 = menu[menu['food']==b3].iloc[0,2]
    cal4 = menu[menu['food']==b4].iloc[0,2]
    cal5 = menu[menu['food']==b5].iloc[0,2]
    cal6 = menu[menu['food']==b6].iloc[0,2]
    cal7 = menu[menu['food']==b7].iloc[0,2]
    cal8 = menu[menu['food']==b8].iloc[0,2]
    name = [b1,b2,b3,b4,b5,b6,b7,b8]
    caleach = [cal1,cal2,cal3,cal4,cal5,cal6,cal7,cal8]
    margin = go.layout.Margin(r=0,l=0,b=0,t=0)
    layout = go.Layout(margin = margin,paper_bgcolor='white')
    color = ['#ff471a', '#ff5c33','#ff704d','#ff8566','#ffff33','#ffff66','#ffff99','#ffffcc']
    fig = go.Figure(data=[go.Pie(labels=name, values=caleach)],layout=layout)
    fig.update(layout_showlegend=False)
    fig.update_traces(marker=dict(colors=color))
    return fig

# callback12: table that can represent the calories for each kind of food chosen
@app.callback(
    Output("table_for_each", "figure"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def draw_table(b1,b2,b3,b4,b5,b6,b7,b8):
    caleach = []
    name = []
    for i in [b1,b2,b3,b4,b5,b6,b7,b8]:
        if i == 'No choice':
            continue
        else:
            caleach.append(menu[menu['food']==i].iloc[0,2])
            name.append(i)
    calpercent = []
    color_inner = ['#ff8566','#ff9980','#ffad99','#ffc2b3','#ffd6cc','#ffebe6','#ffebe6','#ffffff']
    for c in caleach:
        newc = str(round(c*100/sum(caleach),2))+"%"
        calpercent.append(newc)
    if sum(caleach) == 0:
        margin = go.layout.Margin(r=0,l=0,b=70,t=70)
        layout = go.Layout(margin = margin,paper_bgcolor='white')
        fig = go.Figure(data=[go.Table(columnwidth = [240,100,100],header=dict(values=['Menu', 'Calories','percent'],height=35,font_size=15,fill_color='#ff704d'),
                 cells=dict(values=[['No Data'], ['No Data'],['No Data']],height=25,font_size=12,fill_color = [['#ff8566']*3]))],layout=layout)
    else:
        margin = go.layout.Margin(r=0,l=0,b=20,t=20)
        layout = go.Layout(margin = margin,paper_bgcolor='white')
        fig = go.Figure(data=[go.Table(columnwidth = [240,100,100],header=dict(values=['Menu', 'Calories','percent'],height=35,font_size=15,fill_color='#ff704d'),
                    cells=dict(values=[name, caleach,calpercent],height=25,font_size=12,fill_color=[color_inner[:len(name)]*3]))],layout=layout)
    return fig

# callback13: stack bar chart for other nutritions
@app.callback(
    Output("g", "figure"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def draw_stack(b1,b2,b3,b4,b5,b6,b7,b8):
    name = [b1,b2,b3,b4,b5,b6,b7,b8]
    nutriname = ['fat','cholesterol','sodium','carbohydrates','sugar','protein']
    fat = []
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,3])==True:
            fat.append(0)
        else:
            fat.append(menu[menu['food']==i].iloc[0,3])
    cho = []
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,4])==True:
            cho.append(0)
        else:
            cho.append(menu[menu['food']==i].iloc[0,3])
    sodium = []
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,5])==True:
            sodium.append(0)
        else:
            sodium.append(menu[menu['food']==i].iloc[0,5])
    charbo = []
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,6])==True:
            charbo.append(0)
        else:
            charbo.append(menu[menu['food']==i].iloc[0,6])
    sugar = []
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,7])==True:
            sugar.append(0)
        else:
            sugar.append(menu[menu['food']==i].iloc[0,7])
    protein = []
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,8])==True:
            protein.append(0)
        else:
            protein.append(menu[menu['food']==i].iloc[0,8])
    fat_percent = []
    for i in fat:
        fat_percent.append(i/sum(fat))
    cho_percent = []
    for i in cho:
        cho_percent.append(i/sum(cho))
    sodium_percent = []
    for i in sodium:
        sodium_percent.append(i/sum(sodium))
    charbo_percent = []
    for i in charbo:
        charbo_percent.append(i/sum(charbo))
    sugar_percent = []
    for i in sugar:
        sugar_percent.append(i/sum(sugar))
    protein_percent = []
    for i in protein:
        protein_percent.append(i/sum(protein))
    color = ['#ff471a', '#ff5c33','#ff704d','#ff8566','#ffff33','#ffff66','#ffff99','#ffffcc']
    margin = go.layout.Margin(r=0,l=0,b=0,t=35)
    layout = go.Layout(margin = margin,plot_bgcolor='white',paper_bgcolor='white')
    fig = go.Figure(layout=layout,data=[
    go.Bar(marker_color = color[n],name=name[n], y=nutriname, x=[fat_percent[n],cho_percent[n],sodium_percent[n],charbo_percent[n],sugar_percent[n],protein_percent[n]],orientation='h') for n in range(len(name))
])
    fig.update_layout(barmode='stack',showlegend=False,
        xaxis_title="Percentage",
        yaxis_title="Other Nutritions",
        font=dict(
            family="Verdana",
            size=10
            )
        )
    return fig


# callback14: recommendation amount for each kind of nutrition
@app.callback(
    Output("recommendation_amount", "figure"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def draw_table(b1,b2,b3,b4,b5,b6,b7,b8):
    name = [b1,b2,b3,b4,b5,b6,b7,b8]
    total_fat = 0
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,3])==True:
            continue
        else:
            total_fat = total_fat+ menu[menu['food']==i].iloc[0,3]
    total_cho = 0
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,4])==True:
            continue
        else:
            total_cho = total_cho+ menu[menu['food']==i].iloc[0,4]
    total_sodium = 0
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,5])==True:
            continue
        else:
            total_sodium = total_sodium +menu[menu['food']==i].iloc[0,5]
    total_charbo = 0
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,6])==True:
            continue
        else:
            total_charbo = total_charbo+ menu[menu['food']==i].iloc[0,6]
    total_sugar = 0
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,7])==True:
            continue
        else:
            total_sugar = total_sugar + menu[menu['food']==i].iloc[0,7]
    total_protein = 0
    for i in name:
        if pd.isna(menu[menu['food']==i].iloc[0,8])==True:
            continue
        else:
            total_protein = total_protein + menu[menu['food']==i].iloc[0,8]
    nu = ['fat(g)','cholesterol(mg)','sodium(mg)','carbohydrates(g)','sugar(g)','protein(g)']
    nu_new = nu[::-1]
    actual = [total_fat,total_cho,total_sodium,total_charbo,total_sugar,total_protein]
    actual_new = actual[::-1]
    recommend = [70,300,2300,260,31,50]
    recommend_new = recommend[::-1]
    need_data1 = ['Need Data' for i in range(6)]
    need_data2 = ['Need Data' for i in range(6)]
    gap = []
    for i in range(len(actual)):
        if actual[i]-recommend[i]>0:
            gap.append("Need Less!")
        elif actual[i]-recommend[i]==0:
            gap.append("Well down!")
        else:
            gap.append("Need more!")
    gap_new = gap[::-1]
    if sum(actual)==0:
        margin = go.layout.Margin(r=20,l=20,b=0,t=25)
        layout = go.Layout(margin = margin,paper_bgcolor='white')
        fig = go.Figure(data=[go.Table(columnwidth = [180,130,130,150],header=dict(values=['Nutritions','Actual','Recommended','Gap'],height=35,font_size=15,fill_color='#ff704d'),
                 cells=dict(values=[nu_new,need_data1,recommend_new,need_data2],height=25,font_size=12,fill_color = [['#ff8566','#ff9980','#ffad99','#ffc2b3','#ffd6cc','#ffebe6']*4]))],layout=layout)
        fig.update_layout(plot_bgcolor='white')
    else:
        margin = go.layout.Margin(r=20,l=20,b=0,t=25)
        layout = go.Layout(margin = margin,paper_bgcolor='white')
        fig = go.Figure(data=[go.Table(columnwidth = [180,130,200,150],header=dict(values=['Nutritions', 'Actual','Recommended','Gap'],height=35,font_size=15,fill_color='#ff704d'),
                    cells=dict(values=[nu_new,actual_new,recommend_new,gap_new],height=25,font_size=12,fill_color = [['#ff8566','#ff9980','#ffad99','#ffc2b3','#ffd6cc','#ffebe6']*4]))],layout=layout)
        fig.update_layout(plot_bgcolor='white')
    return fig


# the following callback is for the tab2 
# exercise_1 callback
@app.callback(
    dash.dependencies.Output("exercise_1", "options"),
    dash.dependencies.Input("exercise_1", "search_value")
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate
    return [exer for exer in exercise_list if search_value in exer["label"]]

# exercise_2 callback
@app.callback(
    dash.dependencies.Output("exercise_2", "options"),
    dash.dependencies.Input("exercise_2", "search_value")
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate
    return [exer for exer in exercise_list if search_value in exer["label"]]

# exercise_3 callback
@app.callback(
    dash.dependencies.Output("exercise_3", "options"),
    dash.dependencies.Input("exercise_3", "search_value")
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate
    return [exer for exer in exercise_list if search_value in exer["label"]]

# exercise_4 callback
@app.callback(
    dash.dependencies.Output("exercise_4", "options"),
    dash.dependencies.Input("exercise_4", "search_value")
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate
    return [exer for exer in exercise_list if search_value in exer["label"]]

# exercise_5 callback
@app.callback(
    dash.dependencies.Output("exercise_5", "options"),
    dash.dependencies.Input("exercise_5", "search_value")
)
def update_options(search_value):
    if not search_value:
        raise PreventUpdate
    return [exer for exer in exercise_list if search_value in exer["label"]]


# callback6: table that represents the optimal solution: with gurobi optimization
@app.callback(
    dash.dependencies.Output("table_for_each2", "figure"),
    dash.dependencies.Input("weight", "value"),
    dash.dependencies.Input("exercise_1", "value"),
    dash.dependencies.Input("exercise_2", "value"),
    dash.dependencies.Input("exercise_3", "value"),
    dash.dependencies.Input("exercise_4", "value"),
    dash.dependencies.Input("exercise_5", "value"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def draw_table(kg,e1,e2,e3,e4,e5,b1,b2,b3,b4,b5,b6,b7,b8):
    total_cal = menu[menu['food']==b1].iloc[0,2]+ menu[menu['food']==b2].iloc[0,2]+ \
                menu[menu['food']==b3].iloc[0,2]+ menu[menu['food']==b4].iloc[0,2]+ \
                menu[menu['food']==b5].iloc[0,2]+ menu[menu['food']==b6].iloc[0,2]+ \
                menu[menu['food']==b7].iloc[0,2]+ menu[menu['food']==b8].iloc[0,2]
    if total_cal==0:
        margin = go.layout.Margin(r=30,l=0,b=0,t=0)
        layout = go.Layout(margin = margin,paper_bgcolor='white',plot_bgcolor='white')
        fig = go.Figure(data=[go.Table(columnwidth = [100,100,100],header=dict(values=['Exercise', 'Hours','Calories'],height=35,font_size=15,fill_color='#ff704d'),
                    cells=dict(values=[['No Data'], [0],[0]],height=25,font_size=12,fill_color = [['#ffc2b3']*3]))],layout=layout)
    else:
        select_exercise = [e1,e2,e3,e4,e5, 'Total']
        df = pd.read_csv('exercise_data.csv', index_col=0)
        df['cal_per_h'] = df['Calories per kg'] * kg
        sel_df = df.filter(items=select_exercise, axis=0)

        I = sel_df.index
        Ci = sel_df['cal_per_h']
        k = 2

        mod=Model()
        x=mod.addVars(I,vtype=GRB.INTEGER)
        L=mod.addVar(vtype=GRB.INTEGER)
        mod.setObjective(sum(x[i] for i in I),sense=GRB.MINIMIZE)
        for i in I:
            ## min and max for each exercise
            mod.addConstr(x[i]<=8)
            mod.addConstr(x[i]>=1)
            mod.addConstr(x[i]<=L+k)
            mod.addConstr(x[i]>=L)
        mod.addConstr(sum(Ci[i]*x[i] for i in I )>=total_cal)
        mod.setParam('OutputFlag',False)
        mod.optimize()

        solution_df = pd.DataFrame()
        for i in I:
            solution_df = solution_df.append([[i, x[i].x]])
        solution_df.columns = ['exercise', 'hours']
        solution_df = solution_df.merge(sel_df, left_on='exercise', right_on='Category', how = 'inner')
        solution_df['calories'] = round(solution_df['hours']*solution_df['cal_per_h'], 0)
        solution_df = solution_df[['exercise', 'hours', 'calories']]
        solution_df = solution_df.append({'exercise': 'Total', 'hours': mod.ObjVal, 'calories': solution_df['calories'].sum()}, ignore_index=True)

        h1 = solution_df.iloc[0,1]
        h2 = solution_df.iloc[1,1]
        h3 = solution_df.iloc[2,1]
        h4 = solution_df.iloc[3,1]
        h5 = solution_df.iloc[4,1]
        h6 = solution_df.iloc[5,1]

        cal1 = solution_df.iloc[0,2]
        cal2 = solution_df.iloc[1,2]
        cal3 = solution_df.iloc[2,2]
        cal4 = solution_df.iloc[3,2]
        cal5 = solution_df.iloc[4,2]
        cal6 = solution_df.iloc[5,2]
        
        heach = [h1,h2,h3,h4,h5,h6]
        cal = [cal1,cal2,cal3,cal4,cal5,cal6]
        margin = go.layout.Margin(r=30,l=0,b=0,t=0)
        layout = go.Layout(margin = margin,paper_bgcolor='white',plot_bgcolor='white')
        fig = go.Figure(data=[go.Table(columnwidth = [100,100,100],header=dict(values=['Exercise', 'Hours','Calories'],height=35,font_size=15,fill_color='#ff704d'),
                    cells=dict(values=[select_exercise, heach,cal],height=25,font_size=12,fill_color = [['#ffc2b3','#ffc2b3','#ffc2b3','#ffc2b3','#ffc2b3','#ffc2b3']*3]))],layout=layout)
    return fig

# callback7: pie chart for optimal solution: with gurobi optimization
@app.callback(
    dash.dependencies.Output("pie_chart_for_each_choice2", "figure"),
    dash.dependencies.Input("weight", "value"),
    dash.dependencies.Input("exercise_1", "value"),
    dash.dependencies.Input("exercise_2", "value"),
    dash.dependencies.Input("exercise_3", "value"),
    dash.dependencies.Input("exercise_4", "value"),
    dash.dependencies.Input("exercise_5", "value"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def draw_table(kg,e1,e2,e3,e4,e5,b1,b2,b3,b4,b5,b6,b7,b8):
    total_cal = menu[menu['food']==b1].iloc[0,2]+ menu[menu['food']==b2].iloc[0,2]+ \
                menu[menu['food']==b3].iloc[0,2]+ menu[menu['food']==b4].iloc[0,2]+ \
                menu[menu['food']==b5].iloc[0,2]+ menu[menu['food']==b6].iloc[0,2]+ \
                menu[menu['food']==b7].iloc[0,2]+ menu[menu['food']==b8].iloc[0,2]
    if total_cal == 0:
        margin = go.layout.Margin(r=30,l=0,b=0,t=0)
        color = ['#ff3333']
        layout = go.Layout(margin = margin,paper_bgcolor='white',title="Calories Burn per Workout",plot_bgcolor='white')
        fig = go.Figure(data=[go.Pie(values=[0])],layout=layout)
        fig.update(layout_showlegend=False)
        fig.update_traces(marker=dict(colors=color))
    else:
        select_exercise = [e1,e2,e3,e4,e5, 'Total']
        df = pd.read_csv('exercise_data.csv', index_col=0)
        df['cal_per_h'] = df['Calories per kg'] * kg
        sel_df = df.filter(items=select_exercise, axis=0)

        I = sel_df.index
        Ci = sel_df['cal_per_h']
        k = 2

        mod=Model()
        x=mod.addVars(I,vtype=GRB.INTEGER)
        L=mod.addVar(vtype=GRB.INTEGER)
        mod.setObjective(sum(x[i] for i in I),sense=GRB.MINIMIZE)
        for i in I:
            ## min and max for each exercise
            mod.addConstr(x[i]<=8)
            mod.addConstr(x[i]>=1)
            mod.addConstr(x[i]<=L+k)
            mod.addConstr(x[i]>=L)
        mod.addConstr(sum(Ci[i]*x[i] for i in I )>=total_cal)
        mod.setParam('OutputFlag',False)
        mod.optimize()

        solution_df = pd.DataFrame()
        for i in I:
            solution_df = solution_df.append([[i, x[i].x]])
        solution_df.columns = ['exercise', 'hours']
        solution_df = solution_df.merge(sel_df, left_on='exercise', right_on='Category', how = 'inner')
        solution_df['calories'] = round(solution_df['hours']*solution_df['cal_per_h'], 0)
        solution_df = solution_df[['exercise', 'hours', 'calories']]
        solution_df = solution_df.append({'exercise': 'Total', 'hours': mod.ObjVal, 'calories': solution_df['calories'].sum()}, ignore_index=True)

        cal1 = solution_df.iloc[0,2]
        cal2 = solution_df.iloc[1,2]
        cal3 = solution_df.iloc[2,2]
        cal4 = solution_df.iloc[3,2]
        cal5 = solution_df.iloc[4,2]
        
        exercises = [e1,e2,e3,e4,e5]
        cal = [cal1,cal2,cal3,cal4,cal5]
        margin = go.layout.Margin(r=30,l=0,b=0,t=0)
        color = ['#ff3333','#ff6666','#ff9999','#ffcccc','#ffe6e6']
        layout = go.Layout(margin = margin,paper_bgcolor='white',title="Calories Burn per Workout",plot_bgcolor='white')
        fig = go.Figure(data=[go.Pie(labels=exercises, values=cal)],layout=layout)
        fig.update(layout_showlegend=False)
        fig.update_traces(marker=dict(colors=color))
    return fig


# callback8: table for the single exercise
@app.callback(
    dash.dependencies.Output("table_for_single2", "figure"),
    dash.dependencies.Input("weight", "value"),
    dash.dependencies.Input("exercise_1", "value"),
    dash.dependencies.Input("exercise_2", "value"),
    dash.dependencies.Input("exercise_3", "value"),
    dash.dependencies.Input("exercise_4", "value"),
    dash.dependencies.Input("exercise_5", "value"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def draw_table(kg,e1,e2,e3,e4,e5,b1,b2,b3,b4,b5,b6,b7,b8):
    total_cal = menu[menu['food']==b1].iloc[0,2]+ menu[menu['food']==b2].iloc[0,2]+ \
                menu[menu['food']==b3].iloc[0,2]+ menu[menu['food']==b4].iloc[0,2]+ \
                menu[menu['food']==b5].iloc[0,2]+ menu[menu['food']==b6].iloc[0,2]+ \
                menu[menu['food']==b7].iloc[0,2]+ menu[menu['food']==b8].iloc[0,2]
    select_exercise = [e1,e2,e3,e4,e5]
    df = pd.read_csv('exercise_data.csv', index_col=0)
    df['cal_per_h'] = df['Calories per kg'] * kg
    sel_df = df.filter(items=select_exercise, axis=0)
    
    ## this dataframe is used for the single exercise portion
    singleExercise = pd.DataFrame()
    singleExercise['Category'] = sel_df.index
    singleExercise['Hours'] = [total_cal / cal_per_hour for cal_per_hour in sel_df['cal_per_h']]
    singleExercise['Hours_str'] = [f'{hour:.2f} hours' for hour in singleExercise['Hours']]

    df = singleExercise
    margin = go.layout.Margin(r=0,l=0,b=0,t=20)
    layout = go.Layout(margin = margin,paper_bgcolor='white')
    fig = go.Figure(data=[go.Table(columnwidth = [100,100,100],header=dict(values=['Exercise', 'Hours'],height=35,font_size=15,fill_color='#ff704d'),
                 cells=dict(values=[select_exercise, df['Hours_str']],height=25,font_size=12,fill_color = [['#ffc2b3','#ffc2b3','#ffc2b3','#ffc2b3','#ffc2b3']*2]))],layout=layout)
    return fig


# callback9: bar chart for the single exericise
@app.callback(
    dash.dependencies.Output("bar_chart_for_each2", "figure"),
    dash.dependencies.Input("weight", "value"),
    dash.dependencies.Input("exercise_1", "value"),
    dash.dependencies.Input("exercise_2", "value"),
    dash.dependencies.Input("exercise_3", "value"),
    dash.dependencies.Input("exercise_4", "value"),
    dash.dependencies.Input("exercise_5", "value"),
    Input("brand1food1", "value"),
    Input("brand1food2", "value"),
    Input("brand1food3", "value"),
    Input("brand1food4", "value"),
    Input("brand2food1", "value"),
    Input("brand2food2", "value"),
    Input("brand2food3", "value"),
    Input("brand2food4", "value")
)
def draw_bar(kg,e1,e2,e3,e4,e5,b1,b2,b3,b4,b5,b6,b7,b8):
    total_cal = menu[menu['food']==b1].iloc[0,2]+ menu[menu['food']==b2].iloc[0,2]+ \
                menu[menu['food']==b3].iloc[0,2]+ menu[menu['food']==b4].iloc[0,2]+ \
                menu[menu['food']==b5].iloc[0,2]+ menu[menu['food']==b6].iloc[0,2]+ \
                menu[menu['food']==b7].iloc[0,2]+ menu[menu['food']==b8].iloc[0,2]
    select_exercise = [e1,e2,e3,e4,e5]
    df = pd.read_csv('exercise_data.csv', index_col=0)
    df['cal_per_h'] = df['Calories per kg'] * kg
    sel_df = df.filter(items=select_exercise, axis=0)
    
    ## this dataframe is used for the single exercise portion
    singleExercise = pd.DataFrame()
    singleExercise['Category'] = sel_df.index
    singleExercise['Hours'] = [total_cal / cal_per_hour for cal_per_hour in sel_df['cal_per_h']]
    singleExercise['Hours_str'] = [f'{hour:.2f} hours' for hour in singleExercise['Hours']]
    cate = list(sel_df.index)
    hours = [total_cal / cal_per_hour for cal_per_hour in sel_df['cal_per_h']]
    df = singleExercise
    margin = go.layout.Margin(r=0,l=0,b=0,t=0)
    layout = go.Layout(margin = margin,paper_bgcolor='white',plot_bgcolor='white')
    fig = go.Figure([go.Bar(x=cate, y=hours, marker_color='red')],layout=layout)
    fig.update_layout(
        xaxis_title="Workout",
        yaxis_title="Hours",
        font=dict(
            family="Verdana",
            size=10
            )
        )
    return fig


# callback for the tab 3: additional information given
@app.callback(
    Output("food_exercise", "figure"),
    Input("weight", "value"),
    Input("exercise_hour", "value"),
)
def draw_table(w,h):
    result=pd.DataFrame(columns=['Exercise', 'Calories', 'Closest Fast Food'])
    result['Exercise']=df.iloc[:,0]
    result['Calories']=df.loc[:,'Calories per kg']*w*h
    for k in range(len(result)):
        dif_list=[abs(result.loc[k,'Calories']-i) for i in menu['cal']]
        m=dif_list.index(min(dif_list))
        result.loc[k, 'Closest Fast Food']=menu.loc[m,'food']+'--'+menu.loc[m,'brand']
    exercise = list(result.loc[:,'Exercise'])
    calo = list(round(result.loc[:,'Calories'],2))
    # calo = list(result.loc[:,'Calories'])
    clo_food = list(result.loc[:,'Closest Fast Food'])
    margin = go.layout.Margin(r=0,l=0,b=0,t=0)
    layout = go.Layout(margin = margin,paper_bgcolor='#e5f5e0')
    fig = go.Figure(data=[go.Table(columnwidth = [240,100,100],header=dict(values=['Exercise', 'Calories', 'Closest Fast Food'],height=30,font_size=14),
                cells=dict(values=[exercise, calo,clo_food],height=25,font_size=12))],layout=layout)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True,port=8053)

Dash app running on http://127.0.0.1:8053/


<ipython-input-6-eb7e44ba211c>:688: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-6-eb7e44ba211c>:691: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-6-eb7e44ba211c>:694: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-6-eb7e44ba211c>:697: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-6-eb7e44ba211c>:700: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-6-eb7e44ba211c>:703: RuntimeWarning:

invalid value encountered in double_scalars



Academic license - for non-commercial use only - expires 2022-10-02
Academic license - for non-commercial use only - expires 2022-10-02
Using license file /Users/zhiyuzhang/gurobi.lic
Using license file /Users/zhiyuzhang/gurobi.lic
Freeing default Gurobi environment
